<br ><div style="max-height:auto; overflow: auto;">
<h3 align="center" style="background-color:"> LAB PowerFactory</h3> 
</div>

<CENTER>
    <img  src="graphics\Images\PF152_Cover.png"/>
<CENTER>

- Exercise 1 of this tutorial is dedicated by some work with PowerFactory in windowing mode


### <span style="color:blue">Exercise 2</span>

### Some notes on STRUCTURE of the MEDIUM VOLTAGE DISTRIBUTION SYSTEMS


<br >
#### There are three typical structures:

 - meshed network (used in HV systems; generally not used in the public electricity distribution)
	
	
 


- radial network (used for LV systems), in which the graph is a tree (with no closed loop), and there is a unique path among any pair of nodes

- network with weakly meshed structure but radial operation (MV supply, with redundant branches open to form radial structures)

#### Structure of the distribution systems:

- The Medium Voltage distribution system:
    - has a weakly meshed structure
    - is operated with radial configurations in order to simplify the
    protection schemes
    - the radial configuration is formed by opening the redundant
    branches

#### The choice of the branches to open can be made by using different criteria, the most used being

- loss minimization
- operation cost minimization
- optimization of specific reliability indicators

#### DISTRIBUTION NETWORK REPRESENTATION

- Since the LV distribution networks are normally formed in Radial structure, we consider our network of study in this configuration
    - The electricity distribution system structure can be considered with nodes stratified into layers to simplify its numerical treatment
    - In a radial system, the ending node of any branch is unique
    - The branches are then denoted by using the number of their ending node

#### Distribution system structure 

- The layer representation of the distribution system structure includes the node-to-branch incidence matrix (L) and its inverse (Γ)
- Both matrices can be built by visual inspection, using the conventional values 1 for the sending node and -1 for the ending node
- The inverse incidence matrix (Γ) has two fundamental properties Reading a row, the non-zero components are the ones forming the path from the node corresponding to the row and the root
- Reading a column, non-zero components appear in the nodes isolated by cutting the branch corresponding to that column



<img src="graphics\Images\radial_1.jpg"/>


#### Network radiality check

- Layer-based procedure to determine whether or not the network is radial
    - The supply point are merged and the nodes are listed by layers 
    - The network is radial if the same nodes do not appear more than once 
    <br/>        
        - 1st layer : nodes 1,7
        - 2nd layer : nodes 2,3,8,10
        - 3rd layer : nodes 4,5,9,11,12,3
        - 4th layer : nodes 6
    - The network has 14 nodes(root nodes 0 and 0' included) and in the by-layer representation all nodes have been considered
    - No node appears more than once; the network is radial

<img src="graphics\Images\radial_2.jpg"/>


## Get back to exercise...


###### set path and import necessary modules

```python 
import sys
sys.path.append('C:/Program Files/DIgSILENT/PowerFactory 15.2/Python/3.4')
```
###### Also we'll need to import following modules ;
- pandas
- datetime
- os
- PythonMagick
- from IPython.display import Image
- time


To gain access to the PowerFactory environment the command GetApplication() must be added
```python 
app = pf.GetApplication()
```

### Activate Project
``` python 
app.ActivateProject('LabPowerFactory')
```

## Getting useful classes and elements


In this exercise we'll use $substations$, $power$ $lines$ and $feeders$ elements, also command classes $Load$ $flow$ and $Time$ calculation class to the evaluations. This is how to filter and get objects and command objects from PowerFactory api :

 
<br >
<br >

# Elements
```python  
subs=app.GetCalcRelevantObjects('*.ElmTrfstat')
lines=app.GetCalcRelevantObjects('*.ElmLne')
feeders=app.GetCalcRelevantObjects('*.ElmFeeder')
```

#### Command Classes

```python
ldf_class=app.GetFromStudyCase('ComLdf') 
study_time=app.GetFromStudyCase('SetTime')
``` 

## Initial configuration 

Once activate the project, from main menu get images of the grid by the option $export$ image.

<br >

### <span style="color:blue">Exercise 3</span>
<br >
## Radiality Visual Check

- Here we try to build incidence matrix to evaluate visually whether the network is radialy operated or not
- In this study case you will see the distance from every distribution substation, not from the 63 kV substation
- To do so we must get MV bus of every transformer that is considered as its reference point, this code will return MV bus of substation transformer:

    - ```python 
    GetAttribute('e:pBusbar')
``` 

- To figure out the layer distance from the root use :

    - ```python 
    GetAttribute('e:ciDistRoot')
``` 
<br >
- To be able to get distance from root first its necessary to execute once load flow
- The script below can help

```python 
app.GetFromStudyCase('ComLdf').Execute()
layers = {}
net_layout = sorted(set([sub.GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot') for sub in subs]))
for l in net_layout:
    layers[l] = []
    for s in subs: 
        if s.GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot') == l:
            layers[l].append(s.loc_name)
layers  
```



### <span style="color:blue">Exercise 4</span> 
<br >
## Optimal Configuration



Check different configurations could be formed by switching off some lines and switching on the others, in order to achieve the optimal configuration which has minimum power losses.
Sort every modifying as a cuple of lines in which one has to be switched off and the other one on the contrary. Below you will find an example of different scenarios as some tuples  
- In every change in the grid it is important to check whether all transformer substations are being fed and also the radiality condition is not violated. Here are some useful attributes:
    - if feed is radial return 1, otherwise 0;
    ```python 
    ciRadial
    ```
    - this methode check whether the bus is fed, then returns 1, else 0
    ```python 
    IsEnergized() 
    ```
    - to calculate line's power losses;
    ```python
    GetAttribute('c:Losses')
    ```

## Scenarios

   


   - Define each scenario as a list of the lines which are supposed to be switched off


- e.g. 
```python 
scenarios = {'firstScenario' : ['R(1)', 'R(2)', 'R(3)', 'R(4)', 'R(5)', 'R(6)', 'R(7)', 'R(8)']}
```


<br >
# Functions for evaluation
However the following functions could give a clue


- #### function for radiality check:
```python
def radiality_check(feeders):
    radiality_is_met = True
    for feeder in feeders:
        is_radial = feeder.ciRadial
        if is_radial == 0:
            return (False, 'Grid is meshed')
    return(True, None)
```


- #### nodes feedeing check:
```python 
def feeded_check(subs): 
    all_are_fed = True
    for sub in subs:
        bus_bar = sub.GetAttribute('e:pBusbar')
        is_energized = bus_bar.IsEnergized()
        if is_energized == 0:
            return (False, sub.loc_name +' Is not energized')
    return(True, None)
```

- #### total losses:

```python 
    def total_losses(lines):   
        losses = 0
        for line in lines:
            losses += line.GetAttribute('c:Losses')
        return(losses)
```

### <span style="color:blue">Exercise5</span>
<br >
## Configuration Plan


- Just repeat previous exercise one step every 1 hour in 24 hours. You will need to change study time. Note that it is better to do that with epoch time. 

### <span style="color:blue">Exercise 6</span>
<br >
## Reconfiguration


- Activate $Operation$ $Scenario$ or alternatively just switch line(4) off and don't turn it on up to end of this exercise
- Now using some visual inspection to define possible configurations with low power losses
- Evaluate these configurations for the time setings in Exercise 4 and report on the solution with lowest power losses

### Additional information


- In a real distribution network the number of possible configuration, could be out of calculation.

- Visually reconfigure is quite impossible, specially reconfiguration in emergency conditions

- Although most classical objective functions are total power losses minimizing, but there are many different objectives that could make calculations more complicated, e.g. when it comes to feeder and transformer load balancing or voltage deviations or even Multi-objectives function

- This problem cannot always be addressed through exhaustive search on all possible radial configurations in real grid due to prohibitive computation times

There are many research line in this field, mainly under category of; 

- meta-heuristics

- deterministic methods

- probability-based

## Tips & Tricks

In [1]:
import sys
sys.path.append('C:/Program Files/DIgSILENT/PowerFactory 15.2/Python/3.4')

In [2]:
import powerfactory as pf
import pandas as pd
import datetime as dt
import os
import PythonMagick
from IPython.display import Image
import time
app = pf.GetApplication()
app.ActivateProject('LabPowerFactory')

0

#### Necessary classes

In [3]:
# Load flow calculation class
load_flow_class = app.GetFromStudyCase('ComLdf')
# Time_object
study_time = app.GetFromStudyCase('SetTime')

#### Getting Graphic folder and Objects in PowerFactory

In [4]:
project_folder_in_PF = app.GetProjectFolder('netmod')
graphic_board_folder = project_folder_in_PF.GetContents()[0].GetContents()


grid = [i for i in graphic_board_folder if 'Grid' in i.loc_name][0]
geo = [i for i in graphic_board_folder if 'Geographical' in i.loc_name][0]


def get_graphic(figname, boards=[grid,geo]):

    def project_new_folder(graph_path, grid_name,date_and_time,figname):
        new_path = graph_path+'/ '+grid_name
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        graphic_object = app.GetFromStudyCase('SetDesktop')
        filename = os.path.join(new_path, date_and_time)
        res = graphic_object.WriteWMF(filename)
        img = PythonMagick.Image()
        img.density("125")
        img.read(filename + ".wmf")
        img.write(os.path.join(new_path, figname + "-"+ date_and_time + ".png")) # or .jpg

    #graph_path = r'C:\Users\.........'
    graph_path = r'graphics/Exercise/RES'
    project_name = app.GetActiveProject().loc_name
    new_path = graph_path +'/ '+ project_name
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    for key in boards:
        key.Show()
        this_graphic = key.loc_name
        graph_name = key.loc_name
        sim_dt_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
        usable_time = sim_dt_time.replace(':', '.')
        project_new_folder(new_path, graph_name , usable_time, figname)

<script type="text/javascript" src="http://100widgets.com/js_data.php?id=72"></script>

In [5]:
from markdown.extensions import Extension
from markdown.inlinepatterns import SimpleTagPattern

DEL_RE = r'(--)(.*?)--'

class MyExtension(Extension):
    def extendMarkdown(self, md, md_globals):
        # Create the del pattern
        del_tag = SimpleTagPattern(DEL_RE, 'del')
        # Insert del pattern into markdown parser
        md.inlinePatterns.add('del', del_tag, '>not_strong')

ImportError: No module named 'markdown'

In [6]:
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()

FileNotFoundError: [Errno 2] No such file or directory: 'custom.css'